In [28]:
import pandas as pd
import numpy as np
import ast
from utils import choice_form, implicatur

In [2]:
df = pd.read_csv("../data/intermediate/df_merge_results_with_pred.csv")

In [3]:
df.head()

,id_seq,task_form,human_response,best_prediction,best_few_shot,has_conclusion
0,1_0_R,IA4,"('Iac', True)","(2, 99.99971389770508, ['Some', 'managers', 'c...","(2, 39.83977138996124, ['Some', 'managers', 'c...",True
1,1_1_R,EA3,"('Eac', True)","(0, 99.99972581863403, ['All', 'divers', 'ling...","(7, 91.39708280563354, ['No', 'linguists', 'di...",True
2,1_2_R,AI2,"('Ica', True)","(3, 99.99973773956299, ['Some', 'skaters', 'cl...","(2, 43.451812863349915, ['Some', 'climbers', '...",True
3,1_3_R,AA1,"('Aac', True)","(0, 99.99972581863403, ['All', 'bankers', 'tea...","(4, 88.54365907609463, ['Some not', 'bankers',...",True
4,1_4_R,OA4,"('Oca', False)","(4, 99.99974966049194, ['Some not', 'opticians...","(5, 91.92997813224792, ['Some not', 'actuaries...",False


In [6]:
# on retourne 1 si l'humain et la machine ont la meme prédiction, sinon 0
def similarity(human_response, prediction):
    human_response = ast.literal_eval(human_response)
    prediction = ast.literal_eval(prediction)
    return 1 if choice_form[prediction[0]] == human_response[0] else 0


In [7]:
df['similarity_simple'] = df[["human_response","best_prediction"]].apply(lambda x: similarity(x[0],x[1]), axis=1)
df['similarity_mnli'] = df[["human_response","best_few_shot"]].apply(lambda x: similarity(x[0],x[1]), axis=1)

In [8]:
def humanright(human_response):
    human_response =  ast.literal_eval(human_response)
    return human_response[1]

In [10]:
df['human_right'] = df.human_response.apply(lambda x: humanright(x))

In [11]:
df.head()

,id_seq,task_form,human_response,best_prediction,best_few_shot,has_conclusion,similarity_simple,similarity_mnli,human_right
0,1_0_R,IA4,"('Iac', True)","(2, 99.99971389770508, ['Some', 'managers', 'c...","(2, 39.83977138996124, ['Some', 'managers', 'c...",True,1,1,True
1,1_1_R,EA3,"('Eac', True)","(0, 99.99972581863403, ['All', 'divers', 'ling...","(7, 91.39708280563354, ['No', 'linguists', 'di...",True,0,0,True
2,1_2_R,AI2,"('Ica', True)","(3, 99.99973773956299, ['Some', 'skaters', 'cl...","(2, 43.451812863349915, ['Some', 'climbers', '...",True,1,0,True
3,1_3_R,AA1,"('Aac', True)","(0, 99.99972581863403, ['All', 'bankers', 'tea...","(4, 88.54365907609463, ['Some not', 'bankers',...",True,1,0,True
4,1_4_R,OA4,"('Oca', False)","(4, 99.99974966049194, ['Some not', 'opticians...","(5, 91.92997813224792, ['Some not', 'actuaries...",False,0,1,False


In [12]:
df_similarity_mnli_true_origin = df[df.human_right==True].groupby(['has_conclusion','task_form']) \
       .agg({'id_seq':'size', 'similarity_mnli':'mean'}) \
       .rename(columns={'id_seq':'count','similarity_mnli':'mean_mnli_human_right'}) \
       .reset_index()

In [13]:
df_similarity_mnli_false = df[df.human_right==False].groupby('task_form') \
       .agg({'similarity_mnli':'mean'}) \
       .rename(columns={'similarity_mnli':'mean_mnli_human_false'}) \
       .reset_index()

In [14]:
df_similarity_simple_true = df[df.human_right==True].groupby('task_form') \
       .agg({'similarity_simple':'mean'}) \
       .rename(columns={'similarity_simple':'mean_simple_human_right'}) \
       .reset_index()

In [15]:
df_similarity_simple_false = df[df.human_right==False].groupby('task_form') \
       .agg({'similarity_simple':'mean'}) \
       .rename(columns={'similarity_simple':'mean_simple_human_false'}) \
       .reset_index()

In [16]:
df_final = df_similarity_mnli_true_origin.merge(df_similarity_mnli_false, on="task_form")
df_final = df_final.merge(df_similarity_simple_true, on="task_form")
df_final = df_final.merge(df_similarity_simple_false, on="task_form")


In [29]:
df_valid = pd.read_csv("../data/results/result_valid_30_01_22.csv")
df_final_valid = df_final[df_final["has_conclusion"]==True].merge(df_valid, on="task_form")
df_final_valid = df_final_valid[[ "task_form","count", "mean_mnli_human_right","mean_mnli_human_false", "mean_simple_human_right", "mean_simple_human_false", "succes_human" ]]
df_final_valid = df_final_valid.merge(df_valid[["task_form","succes_mnli_00","succes_simple_0.9999965" ]],on="task_form")
df_final_valid['implicatur'] = df_final_valid.task_form.apply(lambda x : True if x in implicatur else False)

In [30]:
df_final_valid.head()

,task_form,count,mean_mnli_human_right,mean_mnli_human_false,mean_simple_human_right,mean_simple_human_false,succes_human,succes_mnli_00,succes_simple_0.9999965,implicatur
0,AA1,82,0.804878,0.090909,0.585366,0.090909,0.788462,0.884615,0.798077,False
1,AA2,69,0.318841,0.400000,0.362319,0.533333,0.696970,0.494949,0.454545,False
2,AA4,31,0.000000,0.500000,0.064516,0.371429,0.306931,0.000000,0.148515,True
3,AE1,91,0.703297,0.000000,0.032967,0.000000,0.892157,1.000000,0.068627,False
4,AE2,12,0.000000,0.340909,0.083333,0.011364,0.120000,0.000000,0.010000,True


In [31]:
df_final_valid.to_csv("../data/results/result_similarity_valid.csv")

In [32]:
df_unvalid = pd.read_csv("../data/results/result_unvalid_30_01_22.csv")
df_final_unvalid = df_final[df_final["has_conclusion"]==False].merge(df_unvalid, on="task_form")
df_final_unvalid = df_final_unvalid[[ "task_form","count", "mean_mnli_human_right","mean_mnli_human_false", "mean_simple_human_right", "mean_simple_human_false", "succes_human" ]]

df_final_unvalid = df_final_unvalid.merge(df_unvalid[["task_form","succes_mnli_00","succes_simple_0.9999965" ]],on="task_form")
df_final_unvalid['implicatur'] = df_final_unvalid.task_form.apply(lambda x : True if x in implicatur else False)

In [33]:
df_final_unvalid.head()

,task_form,count,mean_mnli_human_right,mean_mnli_human_false,mean_simple_human_right,mean_simple_human_false,succes_human,succes_mnli_00,succes_simple_0.9999965,implicatur
0,AA3,42,0.0,0.438596,0.0,0.421053,0.424242,0.0,0.030303,False
1,AI1,13,0.0,0.625000,0.0,0.363636,0.128713,0.0,0.000000,False
2,AI3,37,0.0,0.142857,0.0,0.300000,0.345794,0.0,0.009346,False
3,AO1,12,0.0,0.584270,0.0,0.337079,0.118812,0.0,0.000000,False
4,AO2,19,0.0,0.390805,0.0,0.149425,0.179245,0.0,0.000000,False


In [26]:
df_final_unvalid.to_csv("../data/results/result_similarity_unvalid.csv")